In [ ]:
import torch
_ = torch.tensor([0.2126, 0.7152, 0.0722], names=['c'])

<ipython-input-1-32e0d8fb07c9>:2: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at ../c10/core/TensorImpl.h:1758.)
  _ = torch.tensor([0.2126, 0.7152, 0.0722], names=['c'])


In [ ]:
img_t = torch.randn(3, 5, 5) # shape [channels, rows, columns]
weights = torch.tensor([0.2126, 0.7152, 0.0722])

In [ ]:
batch_t = torch.randn(2, 3, 5, 5) # shape [batch, channels, rows, columns]

In [ ]:
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

#RGB 채널은 0번 혹은 1번 차원에 존재 - 뒤에서 세 번째 차원이므로 RGB 채널은 0번 혹은 1번 차원에 존재
#두 경우 모두 뒤에서부터 세 번째 차원이므로 RGB 채널은 -3번 차원에 있는 것으로 일반화 가능

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [ ]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)
img_weights = (img_t * unsqueezed_weights)
batch_weights = (batch_t * unsqueezed_weights)
img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)
batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

In [ ]:
img_gray_weighted_fancy = torch.einsum('...chw,c->...hw', img_t, weights)
batch_gray_weighted_fancy = torch.einsum('...chw,c->...hw', batch_t, weights)
batch_gray_weighted_fancy.shape


torch.Size([2, 5, 5])

In [ ]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])
weights_named

tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [ ]:
img_named =  img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print("img named:", img_named.shape, img_named.names)
print("batch named:", batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [ ]:
double_points = torch.ones(10, 2, dtype=torch.double)
short_points = torch.tensor([[1, 2], [3, 4]], dtype=torch.short)

short_points.dtype


torch.int16

In [ ]:
double_points = torch.zeros(10, 2).double()
short_points = torch.ones(10, 2).short()

In [ ]:
double_points = torch.zeros(10, 2).to(torch.double)
short_points = torch.ones(10, 2).to(dtype=torch.short)

In [ ]:
points_64 = torch.rand(5, dtype=torch.double)  
points_short = points_64.to(torch.short)
points_64 * points_short  

tensor([0., 0., 0., 0., 0.], dtype=torch.float64)

In [ ]:
#텐서 API 실습
a = torch.ones(3, 2)
a_t = torch.transpose(a, 0, 1)

a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [ ]:
a = torch.ones(3, 2)
a_t = a.transpose(0, 1)

a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [ ]:
a = torch.ones(3, 2)

In [ ]:
a.zero_()
a

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [ ]:
#텐서 저장 공간 접근

points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [ ]:
points_storage = points.storage()
points_storage[0]


4.0

In [ ]:
points.storage()[1]

1.0

In [ ]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
second_point.storage_offset()

2

In [ ]:
second_point.size()

torch.Size([2])

In [ ]:
second_point.shape

torch.Size([2])

In [ ]:
points.stride()

(2, 1)

In [ ]:
second_point = points[1]
second_point.size()

torch.Size([2])

In [ ]:
second_point.storage_offset()

2

In [ ]:
second_point.stride()

(1,)

In [ ]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
second_point[0] = 10.0
points

tensor([[ 4.,  1.],
        [10.,  3.],
        [ 2.,  1.]])

In [ ]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1].clone()
second_point[0] = 10.0
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [ ]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [ ]:
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [ ]:
#두 텐서가 같은 공간을 가리키고 있는지
id(points.storage()) == id(points_t.storage())

True

In [ ]:
points.stride()

(2, 1)

In [ ]:
#스트라이드만 다르다
points_t.stride()

(1, 2)

In [ ]:
some_t = torch.ones(3, 4, 5)
transpose_t = some_t.transpose(0, 2)
some_t.shape

#더 높은 차원에서의 전치 연산은?

torch.Size([3, 4, 5])

In [ ]:
transpose_t.shape

torch.Size([5, 4, 3])

In [ ]:
some_t.stride()

(20, 5, 1)

In [ ]:
transpose_t.stride()

(1, 5, 20)

In [ ]:
#인접한 텐서는 이렇게 정의한다 !
points.is_contiguous()

True

In [ ]:
points_t.is_contiguous()

False

In [ ]:
#contiguous 메소드 사용 - 인접하지 않은 텐서를 인접한 텐서로 만들수 있다.
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [ ]:
points_t.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [ ]:
points_t.stride()

(1, 2)

In [ ]:
points_t_cont = points_t.contiguous()
points_t_cont

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [ ]:
points_t_cont.stride()

(3, 1)

In [ ]:
points_t_cont.storage()

#항목을 열 단위로 배치하기 위해 새 저장 공간을 재편성하였다는것에 집중 !


 4.0
 5.0
 2.0
 1.0
 3.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]